In [24]:
#libraries
import numpy as np

## README

Here I'm just doing a bit of wrangling to make different codeblocks that I need for BEAST xml files. At some point this can start to the form the basis of a broader program to fully automate xml file generation without using BEAUti. For now it just generates some chunks that BEAUti won't make right now. 

In [ ]:
#make generalDataType code block
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/indexed-countries-45.tsv','rU') as infile:
    with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/beast/generalDataType_countryList.txt', 'w') as outfile:
        for line in infile:
            if line.startswith('country'):
                continue
            else:
                outfile.write('\t\t')
                outfile.write('<state code="{}"/>'.format(line.split('\t')[0]))
                outfile.write('\n')

In [47]:
#make markov rewards code block

#link countries to an index to ensure proper ordering is preserved
#current file ordering is correct
index_dict = {}
counter = 0
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/indexed-countries-45.tsv','rU') as infile:
    for line in infile:
        if line.startswith('country'):
            continue
        else:
            index_dict[counter] = line.split('\t')[0] #use country, not region
            counter +=1
            
#initialize empty markov rewards matrix
#need the same number of rows and columns as you have demes. I.E. here it needs to be 45 by 45.
reward_matrix = np.zeros((len(index_dict),len(index_dict)))
for i in range(len(index_dict)): #for row in matrix
    for j in range(len(index_dict)): #for column in matrix
        if i == j: #if indexes match, i.e. the countries are the same:
            reward_matrix[i,j] += 1 #that markov reward entry should == 1.
        else:
            continue #all other entries should be 0.

xml_code ='''\t\t\t<parameter id="{country}_reward" value="{values}"/>\n'''
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/beast/markov_rewards_block.txt', 'w') as outfile:
    outfile.write('\t\t<rewards>\n')
    for i in range(len(reward_matrix)):
        outfile.write(xml_code.format(country=index_dict[i], values = ' '.join(str(value) for value in reward_matrix[i])))
    outfile.write('\t\t</rewards>\n')